In [2]:
# =======================================================================
# STEP 1: INSTALL ALL REQUIRED STABLE LIBRARY VERSIONS
# =======================================================================
# This final version includes 'langchainhub', which is required for hub.pull().
!pip install -q -U langchain=="0.1.20" langchain-core=="0.1.52" langchain-google-genai=="1.0.4" langsmith=="0.1.57" langchainhub=="0.1.20"

# =======================================================================
# STEP 2: CONFIGURE ENVIRONMENT
# =======================================================================
import os
from google.colab import userdata

# Set the official environment variable to disable LangSmith tracing.
os.environ["LANGCHAIN_TRACING_V2"] = "false"

# Configure your Google API key
os.environ['GOOGLE_API_KEY'] = userdata.get('GOOGLE_API_KEY')


# =======================================================================
# STEP 3: IMPORT LIBRARIES (WITH THE CORRECTED TYPO)
# =======================================================================
from langchain_google_genai import ChatGoogleGenerativeAI  # <--- THIS IS THE FIX
from langchain_core.tools import tool
from langchain.agents import AgentExecutor, create_react_agent
from langchain import hub

print("✅ All systems go! Setup is finally complete. You can now proceed.")

✅ All systems go! Setup is finally complete. You can now proceed.


In [8]:
# Bulid the "Math and Joke Agent"
# Here we define the logic and tools for the agent
# Tool is a function that agent can use to perform a specific task


from langchain_core.tools import tool

@tool
def add(a: int, b: int) -> int:
    """Adds two numbers."""
    return a + b

@tool
def subtract(a: int, b: int) -> int:
    """Subtracts two numbers."""
    return a - b

@tool
def tell_a_joke() -> str:
    """Tells a random joke."""
    jokes = [
        "Why don't scientists trust atoms? Because they make up everything!",
        "What do you call fake spaghetti? An Impasta!",
        "Why did the scarecrow win an award? Because he was outstanding in his field!"
    ]
    import random
    return random.choice(jokes)

tools = [add, subtract, tell_a_joke]

In [9]:
# Intialize the Gemma LLM. Set up the connection to the Gemma model using "langchain-google-genai"

from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash") # You can also use "gemini-pro"

#from langchain_community.llms import Ollama
#llm = Ollama(model="gemma:2b")

In [10]:
# The final, complete, and correct version of Step 5
from langchain.agents import AgentExecutor, create_react_agent
from langchain import hub

# This JSON-aware prompt is correct.
prompt = hub.pull("hwchase17/react-json")

# The agent creation is correct.
agent = create_react_agent(llm, tools, prompt)

# THE FIX IS HERE: We add the error handling flag to the AgentExecutor.
# This makes the agent robust and allows it to self-correct.
agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=False,
    handle_parsing_errors=True  # <-- THE FINAL FIX
)

print("✅ Agent created with JSON prompt and self-correcting error handling.")

✅ Agent created with JSON prompt and self-correcting error handling.


In [15]:
# agent_executor.invoke({"input": "What is 12 plus 7?"})

{'input': 'What is 12 plus 7?',
 'output': 'Agent stopped due to iteration limit or time limit.'}

In [14]:
# agent_executor.invoke({"input": "Tell me a joke."})

{'input': 'Tell me a joke.',
 'output': 'I am unable to tell you a joke because the provided tools are not functional.'}

In [13]:
# agent_executor.invoke({"input": "What is 25 subtract 11? After you give me the answer, tell me a joke."})

{'input': 'What is 25 subtract 11? After you give me the answer, tell me a joke.',
 'output': "25 subtract 11 is 14.  Why don't scientists trust atoms? Because they make up everything!"}

In [17]:
# gradio library to create a simple web-based GUI
!pip install -q gradio

In [18]:

# All of our agent logic is now inside this single function
def run_agent(user_input):
    import os
    from langchain_google_genai import ChatGoogleGenerativeAI
    from langchain_core.tools import tool
    from langchain.agents import AgentExecutor, create_react_agent
    from langchain import hub

    # 1. Define Tools
    @tool
    def add(a: int, b: int) -> int:
        """Adds two numbers."""
        return a + b

    @tool
    def subtract(a: int, b: int) -> int:
        """Subtracts two numbers."""
        return a - b

    @tool
    def tell_a_joke() -> str:
        """Tells a random joke."""
        jokes = [
            "Why don't scientists trust atoms? Because they make up everything!",
            "What do you call fake spaghetti? An Impasta!",
            "Why did the scarecrow win an award? Because he was outstanding in his field!"
        ]
        import random
        return random.choice(jokes)

    tools = [add, subtract, tell_a_joke]

    # 2. Initialize LLM (assuming API key is already set)
    llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash")

    # 3. Create Agent and Executor
    prompt = hub.pull("hwchase17/react-json")
    agent = create_react_agent(llm, tools, prompt)
    agent_executor = AgentExecutor(
        agent=agent,
        tools=tools,
        verbose=False, # We keep verbose off for a clean user experience
        handle_parsing_errors=True
    )

    # 4. Invoke the agent and get the response
    response = agent_executor.invoke({"input": user_input})

    # 5. Return only the final output string
    return response['output']

print("✅ Agent function is defined and ready.")

✅ Agent function is defined and ready.


In [19]:
# Launch the Gradio Interface

import gradio as gr

# Create the user interface
iface = gr.Interface(
    fn=run_agent,  # The function to call
    inputs=gr.Textbox(lines=2, placeholder="Ask your agent anything..."), # A text box for input
    outputs="text", # The output is simple text
    title="Math and Joke AI Agent",
    description="This agent can perform addition and subtraction, or tell you a joke. Based on Google's Gemini model."
)

# Launch the interface!
iface.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://c17b145b569f24d394.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
# create a requirements.txt file needed for github
%%writefile requirements.txt
langchain=="0.1.20"
langchain-core=="0.1.52"
langchain-google-genai=="1.0.4"
langsmith=="0.1.57"
langchainhub=="0.1.20"
gradio

In [ ]:
# create a readme.md file for github
%%writefile README.md
# Math and Joke AI Agent

This project is a simple AI agent built with Python, LangChain, and Google's Gemini model. The agent can perform basic arithmetic (addition and subtraction) and tell jokes.

## Features

- **Tool Use:** The agent is equipped with tools for adding, subtracting, and telling jokes.
- **Web UI:** A simple web interface is provided using Gradio to easily interact with the agent.

## How to Run

1.  **Open in Colab:** The easiest way to run this project is to open the `.ipynb` notebook in Google Colab.
2.  **Set API Key:** Before running, you must set your Google AI Studio API key. In the Colab notebook, go to the "Secrets" tab (key icon on the left) and add a new secret named `GOOGLE_API_KEY` with your key as the value.
3.  **Run All Cells:** Click `Runtime -> Run all` to install the dependencies and launch the Gradio web interface. A public link will be generated.